In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import glob

import tifffile as tf

import os

path = "/home/patrick/github/data/liftout"

filenames = glob.glob(os.path.join(path, "c3/**/label/*.tif*"), recursive=True)

print(len(filenames))

In [ ]:
for fname in filenames[:10]:
    img = tf.imread(fname)
    plt.imshow(img, cmap="gray")
    plt.show()

In [ ]:
save_path = "/home/patrick/github/data/liftout/training/c/images/"
os.makedirs(save_path, exist_ok=True)
for i, fname in enumerate(filenames):
    
    img = tf.imread(fname)
    new_fname = os.path.join(save_path,f"c{i:05d}.tif")
    # print(new_fname)
    tf.imsave(new_fname, img)


In [ ]:
train_filenames = sorted(glob.glob(os.path.join(save_path, "*.tif")))
for fname in train_filenames:

    img = tf.imread(fname)
    plt.imshow(img, cmap="gray")
    plt.show()

In [ ]:
import zarr

images = zarr.open(tf.imread(os.path.join("/home/patrick/github/data/liftout/training/train", "images", "c*.tif*"), aszarr=True)) 
labels = zarr.open(tf.imread(os.path.join("/home/patrick/github/data/liftout/training/train", "labels","c*.tif*"), aszarr=True))


print(images.shape)
print(labels.shape)


for img, label in zip(images, labels):


    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img, cmap="gray")
    ax[1].imshow(label)
    plt.show()



### Model Testing

In [ ]:
%load_ext autoreload
%autoreload 2 


import glob
import os
import tifffile as tf
import matplotlib.pyplot as plt

images = sorted(glob.glob(os.path.join("/home/patrick/github/data/training/c/images", "*.tif*")))
labels = sorted(glob.glob(os.path.join("/home/patrick/github/data/training/c/labels", "*.tif*")))

for ii, (i, l) in enumerate(zip(images, labels)):
    print(os.path.basename(i), os.path.basename(l))

    img, lbl = tf.imread(i), tf.imread(l)

    fig, ax = plt.subplots(1, 2, figsize=(7, 5))
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Image")
    ax[1].imshow(lbl)
    ax[1].set_title("Label")
    plt.show()
    
    if ii == 2:
        break



In [ ]:
filenames = sorted(glob.glob(os.path.join("/home/patrick/github/data/liftout/training/train/images/", "*.tif*")))

print("files: ", len(filenames))
# filenames.append(*list(sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/images/", "*.tif*")))
print("files: ", len(filenames))
import random
random.shuffle(filenames)


from fibsem.segmentation.model import SegmentationModel

# baseline = "/home/patrick/github/fibsem/fibsem/segmentation/models/model.pt"
# checkpoint_2 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_01_51_n08_model.pt"
# checkpoint_5 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_00_58_n05_model.pt"
# checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_02_27_n10_model.pt"
# checkpoints = [baseline, checkpoint_2, checkpoint_5, checkpoint_10]

# small_model = "/home/patrick/github/fibsem/fibsem/segmentation/models/model2.pt"
# checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_00_10_n10_model.pt"
# checkpoint_15 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_09_22_n15_model.pt"
# checkpoint_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_18_32_n20_model.pt"

# checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_37_32_n10_model.pt"
# checkpoint_15 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_46_48_n15_model.pt"
# checkpoint_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_56_00_n20_model.pt"



# Nov 21
baseline = "/home/patrick/github/fibsem/fibsem/segmentation/models/model2.pt"

checkpoint18_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/c/18/21_11_2022_16_05_55_n10_model.pt"
checkpoint18_15 = "/home/patrick/github/fibsem/fibsem/segmentation/models/c/18/21_11_2022_16_15_51_n15_model.pt"
checkpoint18_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/c/18/21_11_2022_16_25_45_n20_model.pt"

checkpoint34_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/c/34/21_11_2022_16_47_47_n10_model.pt"
checkpoint34_15 = "/home/patrick/github/fibsem/fibsem/segmentation/models/c/34/21_11_2022_16_57_54_n15_model.pt"
checkpoint34_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/c/34/21_11_2022_17_07_52_n20_model.pt"

checkpoints = [baseline, checkpoint18_10, checkpoint18_15, checkpoint18_20, checkpoint34_10, checkpoint34_15, checkpoint34_20]
for i, fname in enumerate(filenames):

    img = tf.imread(fname)

    print(i, fname) 

    fig, ax = plt.subplots(1, len(checkpoints) + 1, figsize=(15, 5))
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Image")


    titles = ["Baseline", "18_10", "18_15", "18_20", "34_10", "34_15", "34_20"]
    for j, (checkpoint, title) in enumerate(zip(checkpoints, titles), 1):
        model = SegmentationModel(checkpoint=checkpoint, num_classes=3)

        mask = model.inference(img)

        ax[j].imshow(mask)
        ax[j].set_title(title)

    save_path = "training/c"
    os.makedirs(save_path, exist_ok=True)
    fig.savefig(os.path.join(save_path, f"{os.path.basename(fname).replace('tif', 'png')}"))

    plt.show()

In [ ]:
%load_ext autoreload
%autoreload 2

import glob
from fibsem.detection import detection

from fibsem.detection.utils import FeatureType

import numpy as np

# create ones image
img = np.ones((1024, 1536, 3))

img[200:800, 500:1000, 0] = 255
img[200:800, 500:1000, 1] = 0
img[200:800, 500:1000, 2] = 0


import matplotlib.pyplot as plt
plt.imshow(img)
plt.show()

lamella_edge = detection.detect_lamella(img, FeatureType.LamellaCentre, mask_radius=180)

print(lamella_edge)


In [ ]:

print("files: ", len(filenames))
# filenames.append(*list(sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/images/", "*.tif*")))
print("files: ", len(filenames))
import random



from fibsem.segmentation.model import SegmentationModel
from fibsem.imaging import masks
import tifffile as tf

In [ ]:
# Nov 21
baseline = "/home/patrick/github/fibsem/fibsem/segmentation/models/model2.pt"
checkpoint18_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/c/18/21_11_2022_16_25_45_n20_model.pt"
checkpoint34_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/c/34/21_11_2022_17_07_52_n20_model.pt"

filenames = sorted(glob.glob(os.path.join("/home/patrick/github/data/liftout/training/train/images/", "*.tif*")))
random.shuffle(filenames)

# checkpoints = [baseline, checkpoint18_20, checkpoint34_20]
model = SegmentationModel(checkpoint=checkpoint34_20, num_classes=3)


# filenames = ["/home/patrick/github/data/liftout/training/train/images/00136.tif"]
for i, fname in enumerate(filenames):

    img = tf.imread(fname)

    print(i, fname) 

    fig, ax = plt.subplots(1, 2, figsize=(15, 7))
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Image")

    k = 1

    mask = model.inference(img)

    color = (255, 0, 0)
    mask_radius = 256
    # lamella_mask, _ = detection.extract_class_pixels(mask, color)
    # mask = masks.apply_circular_mask(lamella_mask, radius=mask_radius)



    # need to check actually if lamella is present


    try:

        centre = detection.detect_lamella_v2(mask, FeatureType.LamellaCentre, color=color, mask_radius=mask_radius)
        left = detection.detect_lamella_v2(mask, FeatureType.LamellaLeftEdge, color=color, mask_radius=mask_radius)
        right = detection.detect_lamella_v2(mask, FeatureType.LamellaRightEdge, color=color, mask_radius=mask_radius)
        # centre = detection.get_mask_point(mask, "centre", "centre")
        # left = detection.get_mask_point(mask, "left", "centre")
        # right = detection.get_mask_point(mask, "right", "centre")
        # upper = detection.get_mask_point(mask, "centre", "upper")
        # lower = detection.get_mask_point(mask, "centre", "lower")
        # top_left = detection.get_mask_point(mask, "left", "upper")
        # top_right = detection.get_mask_point(mask, "right", "upper")
        # bottom_left = detection.get_mask_point(mask, "left", "lower")
        # bottom_right = detection.get_mask_point(mask, "right", "lower")

        ax[k].set_title("Mask")
        ax[k].imshow(mask)
        ax[k].scatter(centre.x, centre.y, color="white", label="centre")
        ax[k].scatter(left.x, left.y, color="green", label="left")
        ax[k].scatter(right.x, right.y, color="blue", label="right")
        # ax[k].scatter(upper.x, upper.y, color="orange", label="upper")
        # ax[k].scatter(lower.x, lower.y, color="purple", label="lower")
        # ax[k].scatter(top_left.x, top_left.y, color="black", label="top left")
        # ax[k].scatter(top_right.x, top_right.y, color="grey", label="top right")
        # ax[k].scatter(bottom_left.x, bottom_left.y, color="pink", label="bottom left")
        # ax[k].scatter(bottom_right.x, bottom_right.y, color="yellow", label="bottom right")
        ax[k].legend()

        # save figure
        save_path = "training/det_c"
        os.makedirs(save_path, exist_ok=True)
        fig.savefig(os.path.join(save_path, f"{os.path.basename(fname).replace('tif', 'png')}"))
    except Exception as e:
        print(e)
    plt.show()

